In [25]:
import index_database as db
from typing import Iterator

In [26]:
alpha = 0.15
iterations_count = 500

In [ ]:
all_pages : list[db.Urls] = []
calculated_ranks : dict[db.Urls, float] = { }
outgoing_links : dict[db.Urls, list[db.Urls]] = { }

In [28]:
def map_items(pages : Iterator[db.Urls]) -> Iterator[tuple[db.Urls, float]]:
    for current_page in pages: 
        links = outgoing_links[current_page]
        rank = calculated_ranks[current_page]
        
        yield current_page, 0

        if len(links) > 0:
            for page in links:
                yield page, rank / len(links)
        else:
            for page in all_pages:
                yield page, rank / len(all_pages)

def shuffle_items(items : Iterator[tuple[db.Urls, float]]) -> Iterator[tuple[db.Urls, list[float]]]:
    sorted_items = sorted(items, key = lambda url : url[0].id)
    buffer = []
    current_index = None

    for (index, item) in sorted_items:
        if(current_index == None):
            buffer = [item]
        elif current_index == index:
            buffer.append(item)
        else:
            yield   (current_index, buffer)
            buffer = [item]
        current_index = index
    yield (current_index, buffer)

def reduce_items(items : Iterator[tuple[db.Urls, list[float]]]) -> Iterator[tuple[db.Urls, float]]:
    for page, ranks in items:
        yield page, alpha / len(all_pages) + (1 - alpha) * sum(ranks)

In [29]:
def do_iteration():
    mapped = map_items(all_pages)
    shuffled = shuffle_items(mapped)
    reduced = reduce_items(shuffled)

    for page, rank in reduced:
        calculated_ranks[page] = rank
    return

In [ ]:
db.connect()

all_pages = list(db.Urls.get_all_pages())
for page in all_pages:
    calculated_ranks[page] = 1 / len(all_pages)
    outgoing_links[page] = list(page.get_all_outgoing_links())

db.close()

In [31]:
for i in range(iterations_count):
    print("Iterationm", i)
    do_iteration()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
keys = sorted(calculated_ranks, key = lambda key : calculated_ranks[key], reverse= True)

for key in keys:
    print("Page:\t", key.url)
    print("Rank:\t", calculated_ranks[key])
    print()

Page:	 http://hadoop.apache.org/
Rank:	 0.001616778626093184

Page:	 http://www.apache.org/
Rank:	 0.0015730695303953078

Page:	 http://wiki.apache.org/hadoop
Rank:	 0.0015730695303953078

Page:	 https://gitbox.apache.org/repos/asf/hadoop.git
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/ClusterSetup.html
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CommandsManual.html
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/Compatibility.html
Rank:	 0.0015730695303953078

Page:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/Downst